<a href="https://colab.research.google.com/github/AndreiPris/Ai-Agent/blob/main/Day%20I/crew_ollama_deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI Agents Bootcamp: Running DeepSeek with CrewAI and Ollama
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vipbasil/aibootcamp/blob/main/Day%20I/crew_ollama_deepseek.ipynb?raw=true)

This notebook is part of the AI Agents Bootcamp (23–27 June 2025) — it shows how to:
- Set up the `Ollama` environment in Colab
- Run `DeepSeek` models locally for use in CrewAI/MAS pipelines
- Prepare agents that use locally-hosted LLMs with memory and tools

## ⚙️ Step 1: Environment Setup
This installs and runs the `ollama` backend and exposes the service via localtunnel tunnel. Make sure to:
- Restart the runtime if needed
- Use the ngrok alternative (if Cloudflare is blocked or throttled)

In [ ]:
%pip install ollama
%pip install colab-xterm

## 🛠️ System Info Tools (Optional)

Installs utilities (`pciutils`, `lshw`) to inspect hardware specs — useful for checking GPU/CPU availability in Colab or local runtime.








In [ ]:
!sudo apt-get update
!sudo apt-get install pciutils lshw

## 📦 Ollama Installation

Downloads and installs Ollama via the official shell script — run this once per environment setup.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

## 🔧 Step 2: Programmatic Model Management and Server Initialization

In this section, we:
- Import the required libraries for managing subprocesses, HTTP requests, and multithreading
- Start the Ollama server programmatically using a background thread
- Pull the required models (`deepseek-r1:7b`, `llama3`) using `ollama pull`
- Optionally include fallback to a smaller model (`deepseek-r1:1.5b`)
- Confirm the list of available models and test that the local Ollama server is running at `localhost:11434`

📌 **Why it matters**: This sets up your local model infrastructure for agent interaction. You'll later reference `localhost:11434` in your agent definitions to connect to these models.


In [ ]:
# Import necessary libraries
import subprocess
import requests
import json
import threading
from pprint import pprint

##  Launching the Ollama Server in Background

Before using any model, we need to start the **Ollama inference server**, which listens by default on `localhost:11434`.

This snippet:
- Defines a Python function `run_ollama()` that launches `ollama serve`
- Starts it in a **background thread**, so the notebook remains interactive
- Allows the server to stay active without blocking further cells

🛠️ **Note**: You only need to run this once per session. If you restart your Colab, re-run this cell before using any models.


In [ ]:
# Start the Ollama server
def run_ollama():
  subprocess.Popen(["ollama", "serve"])
thread = threading.Thread(target=run_ollama)
thread.start()

## 📥 Pulling Models

We download pre-trained models from the Ollama registry:
- `deepseek-r1:7b` – reasoning & code
- `llama3` – general-purpose assistant

In [ ]:
# Download the deepseek-r1:7b distilled model
!ollama pull deepseek-r1:7b
!ollama pull llama3
# If this doesn't work, you can uncomment the below code to download a smaller model- deepseek-r1:1.5b
# !ollama pull deepseek-r1:1.5b

## 🪶 Pulling Lightweight SLMs

These small models are ideal for fast local agents and low-resource environments:
- `phi3:mini`, `tinyllama` – ultra-small general models
- `gemma:2b` – Google's compact chat model
- `deepseek-r1:1.5b` – distilled reasoning model

In [ ]:
!ollama pull phi3:mini
!ollama pull tinyllama
!ollama pull gemma:2b
!ollama pull deepseek-r1:1.5b

## 🔌 Test Ollama Server

Sends a test request to verify the Ollama server is running on `localhost:11434`.

In [ ]:
!curl http://127.0.0.1:11434

## 📄 Check Installed Models

Lists all models currently downloaded and available in your local Ollama environment.

In [ ]:
!ollama list

# 🧠 Starting the CrewAI Section

##Now we define agents using CrewAI, connected to our locally running Ollama models.  
##This enables multi-agent workflows powered by lightweight, self-hosted LLMs.


In [ ]:
# @title 👨‍🦯 Run this cell to hide all warnings (optional)
# Warning control
import warnings
warnings.filterwarnings('ignore')

# To avoid the restart session warning in Colab, exclude the PIL and
# pydevd_plugins packages from being imported. This is fine because
# we didn't execute the code in the kernel session afterward.

# import sys
# sys.modules.pop('PIL', None)

In [ ]:
# @title ⬇️ Install project dependencies by running this cell
%pip install git+https://github.com/joaomdmoura/crewAI.git --quiet
%pip install crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere --quiet
print("---")
%pip show crewAI crewai_tools langchain_openai langchain_groq langchain_anthropic langchain_community cohere

In [ ]:
%pip install -qU langchain-ollama

## 🧩 Step 3: CrewAI Integration


In [ ]:
import os
import json
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process, LLM
from textwrap import dedent
import warnings
warnings.filterwarnings('ignore')

## Define Agents
In CrewAI, agents are autonomous entities designed to perform specific roles and achieve particular goals. Each agent uses a language model (LLM) and may have specialized tools to help execute tasks.

In [ ]:
# @title 🕵🏻 Define your agents
# import os
# import json
# from datetime import datetime, timedelta
# # from crewai import Agent, Task, Crew, Process, LLM
# # from textwrap import dedent
# import warnings
# warnings.filterwarnings('ignore')

from crewai import Agent
from textwrap import dedent
from langchain_ollama import ChatOllama
# Define LLM (OpenAI used here; replace as needed)
from crewai import LLM

llm = LLM(model="ollama/tinyllama:latest", base_url="http://127.0.0.1:11434")

# =============================================================================
# АГЕНТ 1: КОНСУЛЬТАНТ ПО КЛИЕНТАМ
# =============================================================================

customer_consultant = Agent(
    role=dedent("""Customer Service Consultant"""),
    goal=dedent("""Provide respectful, concise, and effective customer consultations for lingerie purchases"""),
    backstory=dedent("""
        Вы — профессиональный консультант премиум-бренда нижнего белья с 10+ лет опыта.
        Ваш стиль общения: уважительный, лаконичный, по существу. Вы быстро понимаете
        потребности клиента и даете четкие рекомендации без лишних слов.
        Максимум 3-4 предложения в ответе. Цените время клиента.
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# =============================================================================
# АГЕНТ 2: ЛОГИСТ
# =============================================================================

logistics_specialist = Agent(
    role=dedent("""Delivery Logistics Coordinator"""),
    goal=dedent("""Calculate optimal delivery routes and provide accurate delivery times"""),
    backstory=dedent("""
        Вы — опытный логист с экспертизой в области доставки товаров по Кишиневу.
        Знаете все районы города, учитываете пробки, оптимальные маршруты.
        Умеете точно рассчитывать время доставки с учетом текущей дорожной ситуации,
        расстояния и загруженности курьеров.
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# =============================================================================
# АГЕНТ 3: КЛАДОВЩИК
# =============================================================================

warehouse_manager = Agent(
    role=dedent("""Inventory Database Manager"""),
    goal=dedent("""Track and manage product inventory across all store locations"""),
    backstory=dedent("""
        Вы — точный и организованный кладовщик, который ведет базу данных всех товаров:
        модели, цвета, размеры, количество на складах разных магазинов.
        Мгновенно находите нужный товар, отслеживаете остатки и точно знаете,
        где какой товар находится и в каком количестве.
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

# =============================================================================
# АГЕНТ 4: МАРКЕТОЛОГ
# =============================================================================

marketing_specialist = Agent(
    role=dedent("""Marketing Campaign Creator"""),
    goal=dedent("""Create compelling promotional materials and customer engagement campaigns"""),
    backstory=dedent("""
        Вы — креативный маркетолог, специализирующийся на индустрии нижнего белья.
        Умеете создавать привлекательные акции, писать цепляющие тексты для email-рассылок,
        социальных сетей и рекламы. Знаете психологию покупателей и умеете мотивировать на покупку.
        Создаете контент, который увеличивает продажи.
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=3,
    llm=llm
)

## Define Tasks
Tasks in CrewAI are specific assignments given to agents, detailing the actions they need to perform to achieve a particular goal. Tasks can have dependencies and context, and can be executed asynchronously to ensure an efficient workflow.

In [ ]:
# @title 📝 Define your tasks
from crewai import Task
from textwrap import dedent

# Task 1: Researcher
task_1 = Task(
    description=dedent("""
        Research recent developments in open-source large language models (LLMs), with a focus on DeepSeek, its capabilities, and recent benchmarks. Identify 2–3 major advantages compared to closed-source models.
    """),
    expected_output=dedent("""
        A structured summary (~200 words) covering:
        - What DeepSeek is and who developed it
        - Key features or innovations
        - At least two comparative strengths of open-source LLMs
    """),
    agent=agent_1,
)

# Task 2: Analyst
task_2 = Task(
    description=dedent("""
        Analyze the research findings about DeepSeek and extract educational implications. Focus on how this technology could benefit educators, students, or institutions using local infrastructure.
    """),
    expected_output=dedent("""
        A short analytical breakdown (~150 words) including:
        - 2–3 practical use cases in education
        - The impact of offline/local LLMs on cost and data privacy
        - Any challenges or limitations to note
    """),
    agent=agent_2,
    context=[task_1],
)

# Task 3: Writer
task_3 = Task(
    description=dedent("""
        Write a clear, engaging blog post summarizing the findings and analysis into a cohesive article titled:
        "How Open-Source AI Like DeepSeek Is Changing Education."
    """),
    expected_output=dedent("""
        A 400–500 word markdown-formatted blog post including:
        - Title and subtitle
        - Introduction
        - Three core paragraphs with bullet points if needed
        - Conclusion and future outlook
    """),
    agent=agent_3,
    context=[task_2],
)


In [ ]:
print("## 👥 Welcome to the DeepSeek Research Crew")
print('-------------------------------------------')

# Input variables for tasks
var_1 = input("🔍 Topic or model to explore (e.g., DeepSeek)?\n")
var_2 = input("🎯 Who is the target audience (e.g., educators, developers)?\n")
var_3 = input("🧠 What’s the intended output format (e.g., article, summary)?\n")

print('-------------------------------------------')
print(f"✔️ Input received:\nModel: {var_1}\nAudience: {var_2}\nOutput Type: {var_3}")

In [ ]:
# @title 🚀 Get your crew to work!
def main():
    # Instantiate your crew with a sequential process
    crew = Crew(
        agents=[agent_1, agent_2, agent_3],
        tasks=[task_1, task_2, task_3],
        verbose=True,  # You can set it to True or False
        # ↑ indicates the verbosity level for logging during execution.
        process=Process.sequential,
        planning_llm=llm
        # ↑ the process flow that the crew will follow (e.g., sequential, hierarchical).
    )

    inputs = {
    "var_1": var_1,
    "var_2": var_2,
    "var_3": var_3
    }

    result = crew.kickoff(inputs=inputs)
    print("\n\n########################")
    print("## Here is your custom crew run result:")
    print("########################\n")
    print(result)

    return result

if __name__ == "__main__":
  result = main()

In [ ]:
# @title 🖥️ Display the results of your crew as markdown
from IPython.display import display, Markdown

markdown_text = result.raw  # Adjust this based on the actual attribute

# Display the markdown content
display(Markdown(markdown_text))

In [ ]:
# 🩱 AI Agents for Lingerie Brand - Google Colab Version (English)
# Adapted for bootcamp environment with phi3:mini model

import os
import json
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process, LLM
from textwrap import dedent
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# LLM SETUP FOR ALL AGENTS
# =============================================================================

# Using phi3:mini model for better performance
llm = LLM(model="ollama/phi3:mini", base_url="http://127.0.0.1:11434")

# Альтернативно можно использовать другие модели:
# llm = LLM(model="ollama/tinyllama:latest", base_url="http://127.0.0.1:11434")
# llm = LLM(model="ollama/phi3:mini", base_url="http://127.0.0.1:11434")
# llm = LLM(model="ollama/deepseek-r1:1.5b", base_url="http://127.0.0.1:11434")

print("🤖 Initializing AI Agents for Lingerie Brand...")
print("🔗 Connecting to Ollama:", "http://127.0.0.1:11434")
print("🧠 Using model: phi3:mini")

# =============================================================================
# AGENT 1: CUSTOMER CONSULTANT
# =============================================================================

customer_consultant = Agent(
    role="Customer Service Consultant",
    goal="Provide respectful, concise, and effective customer consultations for lingerie purchases",
    backstory=dedent("""
        You are a professional consultant for a premium lingerie brand with 10+ years of experience.
        Your communication style is: respectful, concise, and to the point. You quickly understand
        customer needs and provide clear recommendations without unnecessary words.
        Maximum 3-4 sentences in your response. You value the customer's time.

        You know:
        - Bra sizing basics: band size + cup size
        - Common fitting issues and solutions
        - Different bra styles and their purposes
        - When to ask clarifying questions
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=2,
    llm=llm
)

# =============================================================================
# AGENT 2: LOGISTICS SPECIALIST
# =============================================================================

logistics_specialist = Agent(
    role="Delivery Logistics Coordinator",
    goal="Calculate optimal delivery routes and provide accurate delivery times",
    backstory=dedent("""
        You are an experienced logistics coordinator with expertise in delivery services in Chisinau.
        You know all city districts, account for traffic jams, and optimal routes.
        You can accurately calculate delivery times considering current traffic situation,
        distance, and courier availability.

        You understand:
        - Different delivery zones and their characteristics
        - Traffic patterns throughout the day
        - Courier capacity and vehicle types
        - Cost calculation based on distance and zone
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=2,
    llm=llm
)

# =============================================================================
# AGENT 3: WAREHOUSE MANAGER
# =============================================================================

warehouse_manager = Agent(
    role="Inventory Database Manager",
    goal="Track and manage product inventory across all store locations",
    backstory=dedent("""
        You are a precise and organized warehouse manager who maintains a database of all products:
        models, colors, sizes, quantities at different store warehouses.
        You can instantly find the right product, track inventory levels, and know exactly
        where each item is located and in what quantity.

        You manage:
        - Product catalog with all variants
        - Inventory levels across multiple stores
        - Stock reservations and availability
        - Store locations and addresses
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=2,
    llm=llm
)

# =============================================================================
# AGENT 4: MARKETING SPECIALIST
# =============================================================================

marketing_specialist = Agent(
    role="Marketing Campaign Creator",
    goal="Create compelling promotional materials and customer engagement campaigns",
    backstory=dedent("""
        You are a creative marketer specializing in the lingerie industry.
        You can create attractive promotions, write engaging texts for email campaigns,
        social media, and advertising. You understand customer psychology and know how to motivate purchases.
        You create content that increases sales.

        Your expertise includes:
        - Understanding lingerie market psychology
        - Creating tasteful and appealing campaigns
        - Email marketing and social media content
        - Seasonal trends and promotional strategies
    """),
    allow_delegation=False,
    verbose=True,
    max_iter=2,
    llm=llm
)

# =============================================================================
# MOCK DATABASE - SIMPLIFIED DATABASE FOR COLAB
# =============================================================================

# Since Colab doesn't have SQLite, we use dictionaries
PRODUCTS_DB = {
    "1": {"model": "Comfort Plus", "brand": "Intimissimi", "color": "Black", "size": "75B", "price": 89.90},
    "2": {"model": "Comfort Plus", "brand": "Intimissimi", "color": "Beige", "size": "75B", "price": 89.90},
    "3": {"model": "Comfort Plus", "brand": "Intimissimi", "color": "Black", "size": "75C", "price": 89.90},
    "4": {"model": "Elegant", "brand": "Victoria Secret", "color": "Red", "size": "75B", "price": 159.90},
    "5": {"model": "Sport Active", "brand": "Nike", "color": "Gray", "size": "M", "price": 79.90},
    "6": {"model": "Bridal", "brand": "La Perla", "color": "White", "size": "70B", "price": 299.90},
}

INVENTORY_DB = {
    "1": {"product_id": "1", "store": "Central Store", "quantity": 15, "address": "Stefan cel Mare 126 St, Chisinau"},
    "2": {"product_id": "1", "store": "Mall Shopping", "quantity": 8, "address": "Malldova TC, Chisinau"},
    "3": {"product_id": "2", "store": "Central Store", "quantity": 12, "address": "Stefan cel Mare 126 St, Chisinau"},
    "4": {"product_id": "5", "store": "Central Store", "quantity": 25, "address": "Stefan cel Mare 126 St, Chisinau"},
    "5": {"product_id": "6", "store": "Botanica Boutique", "quantity": 3, "address": "Dacia 47 St, Chisinau"},
}

DELIVERY_ZONES = {
    "Center": {"time": 20, "cost": 25.0, "traffic_multiplier": 1.2},
    "Botanica": {"time": 35, "cost": 30.0, "traffic_multiplier": 1.1},
    "Riscani": {"time": 45, "cost": 35.0, "traffic_multiplier": 1.3},
    "Buiucani": {"time": 40, "cost": 35.0, "traffic_multiplier": 1.2},
}

# =============================================================================
# TASKS FOR EACH AGENT
# =============================================================================

def create_consultation_task(customer_query):
    """Creates customer consultation task"""
    return Task(
        description=dedent(f"""
            A customer asked: "{customer_query}"

            Response requirements:
            - Maximum 3-4 sentences
            - Respectful tone
            - Specific recommendations
            - No unnecessary words
            - If additional information is needed - ask 1 clarifying question

            Remember you are a professional lingerie consultant helping customers find the right fit and style.
        """),
        expected_output=dedent("""
            Brief professional response including:
            - Greeting (if first contact)
            - Specific recommendation or solution
            - Clarifying question (if needed)
            - Polite closing
        """),
        agent=customer_consultant
    )

def create_inventory_task(product_request):
    """Creates inventory check task"""

    # Search in mock database
    found_products = []
    for pid, product in PRODUCTS_DB.items():
        if any(word.lower() in product["model"].lower() or
               word.lower() in product["color"].lower() or
               word.lower() in product["size"].lower()
               for word in product_request.split()):
            # Search in inventory
            for iid, inventory in INVENTORY_DB.items():
                if inventory["product_id"] == pid:
                    found_products.append({
                        "model": product["model"],
                        "color": product["color"],
                        "size": product["size"],
                        "price": product["price"],
                        "store": inventory["store"],
                        "quantity": inventory["quantity"],
                        "address": inventory["address"]
                    })

    inventory_data = json.dumps(found_products, ensure_ascii=False, indent=2)

    return Task(
        description=dedent(f"""
            Check product availability for request: "{product_request}"

            Database results:
            {inventory_data}

            Provide information about:
            - Availability by stores
            - Exact quantities
            - Store addresses
            - Prices
            - Reservation possibility
        """),
        expected_output=dedent("""
            Complete product availability information:
            - List of found products
            - Store availability with addresses
            - Quantities in stock
            - Prices
            - Store selection recommendations
        """),
        agent=warehouse_manager
    )

def create_logistics_task(delivery_request):
    """Creates delivery planning task"""

    # Determine delivery zone
    zone = "Center"  # default
    if "botanica" in delivery_request.lower() or "dacia" in delivery_request.lower():
        zone = "Botanica"
    elif "riscani" in delivery_request.lower():
        zone = "Riscani"
    elif "buiucani" in delivery_request.lower():
        zone = "Buiucani"

    zone_info = DELIVERY_ZONES.get(zone, DELIVERY_ZONES["Center"])
    current_time = datetime.now()

    # Calculate time considering traffic
    hour = current_time.hour
    multiplier = zone_info["traffic_multiplier"]
    if 8 <= hour <= 10 or 17 <= hour <= 19:  # rush hour
        multiplier *= 1.5

    delivery_time = int(zone_info["time"] * multiplier)
    arrival_time = current_time + timedelta(minutes=delivery_time)

    logistics_data = {
        "zone": zone,
        "base_time": zone_info["time"],
        "traffic_multiplier": multiplier,
        "delivery_time": delivery_time,
        "arrival_time": arrival_time.strftime('%H:%M'),
        "cost": zone_info["cost"],
        "current_time": current_time.strftime('%H:%M')
    }

    return Task(
        description=dedent(f"""
            Calculate delivery for order: "{delivery_request}"

            Calculated data:
            {json.dumps(logistics_data, ensure_ascii=False, indent=2)}

            Consider:
            - Current time and traffic
            - Distance to delivery zone
            - Road congestion
            - Order priority
        """),
        expected_output=dedent("""
            Complete delivery plan including:
            - Delivery zone
            - Exact arrival time
            - Delivery cost
            - Traffic and road situation considerations
            - Order timing recommendations
        """),
        agent=logistics_specialist
    )

def create_marketing_task(campaign_brief):
    """Creates marketing campaign task"""
    return Task(
        description=dedent(f"""
            Create marketing campaign: "{campaign_brief}"

            Develop:
            - Attractive offer
            - Catchy headline
            - Text for different channels (email, social media)
            - Campaign conditions
            - Call-to-action

            Consider lingerie sales specifics:
            - Topic sensitivity
            - Customer psychology
            - Seasonality and trends
        """),
        expected_output=dedent("""
            Ready marketing campaign including:
            - Campaign headline
            - Main offer text
            - Participation conditions
            - Call to action
            - Adaptation for email and Instagram
        """),
        agent=marketing_specialist
    )

# =============================================================================
# READY SCENARIOS FOR TESTING
# =============================================================================

def scenario_customer_consultation(query):
    """Scenario 1: Customer consultation"""
    print(f"\n📞 CUSTOMER CONSULTATION:")
    print(f"Query: {query}")
    print("-" * 50)

    task = create_consultation_task(query)
    crew = Crew(
        agents=[customer_consultant],
        tasks=[task],
        verbose=False,
        process=Process.sequential,
        planning_llm=llm
    )

    result = crew.kickoff()
    print(f"🤖 Consultant's response:\n{result}")
    return result

def scenario_inventory_check(product_query):
    """Scenario 2: Inventory check"""
    print(f"\n📦 INVENTORY CHECK:")
    print(f"Search: {product_query}")
    print("-" * 50)

    task = create_inventory_task(product_query)
    crew = Crew(
        agents=[warehouse_manager],
        tasks=[task],
        verbose=False,
        process=Process.sequential,
        planning_llm=llm
    )

    result = crew.kickoff()
    print(f"🤖 Warehouse manager's report:\n{result}")
    return result

def scenario_delivery_planning(delivery_query):
    """Scenario 3: Delivery planning"""
    print(f"\n🚚 DELIVERY PLANNING:")
    print(f"Address: {delivery_query}")
    print("-" * 50)

    task = create_logistics_task(delivery_query)
    crew = Crew(
        agents=[logistics_specialist],
        tasks=[task],
        verbose=False,
        process=Process.sequential,
        planning_llm=llm
    )

    result = crew.kickoff()
    print(f"🤖 Delivery plan:\n{result}")
    return result

def scenario_marketing_campaign(campaign_query):
    """Scenario 4: Marketing campaign"""
    print(f"\n📢 MARKETING CAMPAIGN:")
    print(f"Brief: {campaign_query}")
    print("-" * 50)

    task = create_marketing_task(campaign_query)
    crew = Crew(
        agents=[marketing_specialist],
        tasks=[task],
        verbose=False,
        process=Process.sequential,
        planning_llm=llm
    )

    result = crew.kickoff()
    print(f"🤖 Marketing campaign:\n{result}")
    return result

def scenario_complex_order():
    """Scenario 5: Complex order processing"""
    print(f"\n🛍️ COMPLEX ORDER PROCESSING")
    print("-" * 50)

    customer_query = "Looking for a comfortable sports bra in gray color, size 75B, with delivery to Botanica district"

    # Create tasks for all agents
    consultation_task = create_consultation_task(customer_query)
    inventory_task = create_inventory_task("sports gray 75B")
    delivery_task = create_logistics_task("delivery to Botanica, Dacia 47 street")

    # Create team of all agents
    crew = Crew(
        agents=[customer_consultant, warehouse_manager, logistics_specialist],
        tasks=[consultation_task, inventory_task, delivery_task],
        verbose=True,
        process=Process.sequential,
        planning_llm=llm
    )

    result = crew.kickoff()
    print(f"🤖 Complex order processing:\n{result}")
    return result

# =============================================================================
# INTERACTIVE MENU FOR COLAB
# =============================================================================

def run_interactive_demo():
    """Interactive agent demonstration"""
    print("🩱 AI AGENTS FOR LINGERIE BRAND")
    print("=" * 60)
    print("Welcome to the AI agents demonstration!")
    print("\nAvailable scenarios:")
    print("1. 👩‍💼 Customer consultation")
    print("2. 📦 Product inventory check")
    print("3. 🚚 Delivery planning")
    print("4. 📢 Marketing campaign creation")
    print("5. 🛍️ Complex order processing")
    print("6. 🧪 Run all tests")

    return True

# =============================================================================
# PRESET TESTS
# =============================================================================

def run_all_tests():
    """Run all test scenarios"""
    print("🧪 RUNNING ALL TEST SCENARIOS")
    print("=" * 60)

    # Test 1: Consultation
    scenario_customer_consultation(
        "Hello! I'm looking for a comfortable wireless bra for everyday wear, size 75B, preferably in dark colors"
    )

    # Test 2: Inventory
    scenario_inventory_check("Comfort Plus black 75B")

    # Test 3: Delivery
    scenario_delivery_planning("Stefan cel Mare 126 street, Chisinau")

    # Test 4: Marketing
    scenario_marketing_campaign(
        "Spring promotion: 25% discount on entire sports collection, target audience - active women 25-40 years old"
    )

    # Test 5: Complex
    scenario_complex_order()

    print("\n" + "=" * 60)
    print("✅ ALL TESTS COMPLETED!")
    print("🚀 Agents are ready to work!")

# =============================================================================
# QUICK TESTS FOR INDIVIDUAL AGENTS
# =============================================================================

def quick_test_consultant():
    """Quick consultant test"""
    print("⚡ QUICK CONSULTANT TEST")
    print("=" * 40)

    result = scenario_customer_consultation(
        "Hello! What bra size would fit me if my underbust measurement is 75 cm?"
    )
    return result

def quick_test_warehouse():
    """Quick warehouse test"""
    print("⚡ QUICK WAREHOUSE TEST")
    print("=" * 40)

    result = scenario_inventory_check("black bra size 75B")
    return result

def quick_test_logistics():
    """Quick logistics test"""
    print("⚡ QUICK LOGISTICS TEST")
    print("=" * 40)

    result = scenario_delivery_planning("Dacia 47 street, Botanica district")
    return result

def quick_test_marketing():
    """Quick marketing test"""
    print("⚡ QUICK MARKETING TEST")
    print("=" * 40)

    result = scenario_marketing_campaign(
        "Valentine's Day campaign: 20% discount on romantic lingerie sets"
    )
    return result

# =============================================================================
# SYSTEM STATUS CHECK
# =============================================================================

def system_check():
    """Check system status"""
    print("🔍 SYSTEM STATUS CHECK")
    print("=" * 40)

    print(f"✅ LLM model: {llm.model}")
    print(f"✅ Base URL: {llm.base_url}")

    print(f"📦 Products in database: {len(PRODUCTS_DB)}")
    print(f"🏪 Inventory records: {len(INVENTORY_DB)}")
    print(f"🗺️ Delivery zones: {len(DELIVERY_ZONES)}")

    print("\n📋 Available products:")
    for pid, product in PRODUCTS_DB.items():
        print(f"  - {product['model']} {product['color']} {product['size']} (${product['price']})")

    print("\n🏬 Stores:")
    stores = set()
    for inv in INVENTORY_DB.values():
        stores.add(f"{inv['store']} ({inv['address']})")
    for store in stores:
        print(f"  - {store}")

# Initialization completed
print("✅ Agents successfully initialized!")
print("📚 To run, use these functions:")
print("   - run_interactive_demo() - interactive menu")
print("   - run_all_tests() - run all tests")
print("   - quick_test_consultant() - test consultant")
print("   - quick_test_warehouse() - test warehouse manager")
print("   - quick_test_logistics() - test logistics")
print("   - quick_test_marketing() - test marketing")
print("   - system_check() - check system status")

In [ ]:
# =========================================================================
# INTERACTIVE CUSTOMER CONSULTANT TESTER
# =========================================================================

import time
from datetime import datetime

print("🤖 INTERACTIVE CUSTOMER CONSULTANT TESTER")
print("=" * 60)

# =========================================================================
# SIMPLE INTERACTIVE MODE
# =========================================================================

def chat_with_consultant():
    """Simple chat interface with the consultant"""
    print("\n💬 CHAT WITH CUSTOMER CONSULTANT")
    print("-" * 40)
    print("Type your questions and get instant responses!")
    print("Type 'quit', 'exit', or 'stop' to end the session")
    print("Type 'help' for example questions")
    print("-" * 40)

    conversation_count = 0

    while True:
        try:
            # Get user input
            user_question = input("\n👤 Your question: ").strip()

            # Check for exit commands
            if user_question.lower() in ['quit', 'exit', 'stop', 'bye']:
                print("👋 Thank you for testing the consultant! Goodbye!")
                break

            # Check for empty input
            if not user_question:
                print("⚠️ Please enter a question or type 'quit' to exit")
                continue

            # Help command
            if user_question.lower() == 'help':
                show_example_questions()
                continue

            # Process the question
            conversation_count += 1
            print(f"\n🤖 Consultant (Response #{conversation_count}):")
            print("-" * 30)

            start_time = time.time()
            response = scenario_customer_consultation(user_question)
            end_time = time.time()

            print(f"\n⏱️ Response time: {end_time - start_time:.2f} seconds")
            print("=" * 50)

        except KeyboardInterrupt:
            print("\n\n👋 Session ended by user. Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again or type 'quit' to exit")

# =========================================================================
# GUIDED INTERACTIVE MODE
# =========================================================================

def guided_consultant_test():
    """Guided testing with categories"""
    print("\n🎯 GUIDED CONSULTANT TESTING")
    print("-" * 40)

    categories = {
        "1": "Bra Sizing Questions",
        "2": "Style and Fit Questions",
        "3": "Problem Solving",
        "4": "Special Needs",
        "5": "Product Recommendations",
        "6": "Care and Maintenance",
        "7": "Free Form Question"
    }

    while True:
        print("\n📋 Choose a category to test:")
        for key, value in categories.items():
            print(f"   {key}. {value}")
        print("   0. Exit")

        try:
            choice = input("\nEnter category number (0-7): ").strip()

            if choice == "0":
                print("👋 Goodbye!")
                break
            elif choice in categories:
                if choice == "7":
                    # Free form
                    question = input("\n👤 Enter your question: ").strip()
                else:
                    # Guided question for category
                    question = get_guided_question(choice)

                if question:
                    print(f"\n🤖 Testing: {question}")
                    print("-" * 50)
                    start_time = time.time()
                    response = scenario_customer_consultation(question)
                    end_time = time.time()
                    print(f"\n⏱️ Response time: {end_time - start_time:.2f} seconds")

                    # Ask for rating
                    rate_response()
            else:
                print("⚠️ Invalid choice. Please enter 0-7")

        except KeyboardInterrupt:
            print("\n\n👋 Session ended. Goodbye!")
            break

# =========================================================================
# QUESTION HELPERS
# =========================================================================

def show_example_questions():
    """Show example questions for inspiration"""
    print("\n💡 EXAMPLE QUESTIONS:")
    print("-" * 30)

    examples = {
        "Sizing": [
            "What size should I wear if my underbust is 75cm?",
            "I usually wear 34B but it feels tight. What should I try?",
            "How do I convert US size 34C to European sizing?"
        ],
        "Style": [
            "What bra works best under a tight white t-shirt?",
            "I need a strapless bra for my wedding dress",
            "What's the most supportive sports bra for running?"
        ],
        "Problems": [
            "My bra straps keep falling down. How do I fix this?",
            "I get red marks from my underwire. What's wrong?",
            "The center gore doesn't lie flat against my chest"
        ],
        "Special": [
            "I'm pregnant and my size keeps changing. Help!",
            "I need comfortable nursing bras",
            "What's good for a teenager's first bra?"
        ]
    }

    for category, questions in examples.items():
        print(f"\n🏷️ {category}:")
        for q in questions:
            print(f"   • {q}")

def get_guided_question(category_num):
    """Get guided question based on category"""
    prompts = {
        "1": "Enter a bra sizing question (measurements, fit issues, size conversions): ",
        "2": "Enter a style question (what bra for what outfit/occasion): ",
        "3": "Describe a bra fitting problem you're experiencing: ",
        "4": "Ask about special needs (pregnancy, nursing, medical, etc.): ",
        "5": "Ask for product recommendations (brands, types, budget): ",
        "6": "Ask about bra care and maintenance: "
    }

    prompt = prompts.get(category_num, "Enter your question: ")
    return input(f"\n👤 {prompt}").strip()

def rate_response():
    """Allow user to rate the response"""
    try:
        print("\n⭐ How would you rate this response?")
        print("1 = Poor, 2 = Fair, 3 = Good, 4 = Very Good, 5 = Excellent")
        rating = input("Rating (1-5) or press Enter to skip: ").strip()

        if rating in ['1', '2', '3', '4', '5']:
            rating_text = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent'][int(rating)-1]
            print(f"✅ Thank you! You rated this response: {rating}/5 ({rating_text})")

            if int(rating) <= 2:
                feedback = input("💬 What could be improved? (optional): ").strip()
                if feedback:
                    print(f"📝 Feedback noted: {feedback}")
    except:
        pass

# =========================================================================
# BATCH TESTING MODE
# =========================================================================

def batch_test_mode():
    """Test multiple questions in batch"""
    print("\n📊 BATCH TESTING MODE")
    print("-" * 40)
    print("Enter multiple questions (one per line)")
    print("Type 'END' on a new line when finished")
    print("-" * 40)

    questions = []

    while True:
        question = input("Question: ").strip()
        if question.upper() == 'END':
            break
        if question:
            questions.append(question)

    if not questions:
        print("⚠️ No questions entered")
        return

    print(f"\n🚀 Testing {len(questions)} questions...")
    print("=" * 50)

    total_time = 0

    for i, question in enumerate(questions, 1):
        print(f"\n📝 Question {i}/{len(questions)}: {question}")
        print("-" * 40)

        start_time = time.time()
        response = scenario_customer_consultation(question)
        end_time = time.time()

        response_time = end_time - start_time
        total_time += response_time

        print(f"⏱️ Time: {response_time:.2f}s")
        print("=" * 50)

    print(f"\n📈 BATCH RESULTS:")
    print(f"   Total questions: {len(questions)}")
    print(f"   Total time: {total_time:.2f}s")
    print(f"   Average time: {total_time/len(questions):.2f}s per question")

# =========================================================================
# CONVERSATION MODE
# =========================================================================

def conversation_mode():
    """Extended conversation with context"""
    print("\n💭 CONVERSATION MODE")
    print("-" * 40)
    print("Have an extended conversation with the consultant")
    print("The agent will remember the context of your conversation")
    print("Type 'quit' to end the conversation")
    print("-" * 40)

    conversation_history = []

    while True:
        try:
            question = input(f"\n👤 You (message #{len(conversation_history)+1}): ").strip()

            if question.lower() in ['quit', 'exit', 'stop']:
                break

            if not question:
                continue

            # Add context from previous conversation
            if conversation_history:
                context_question = f"Previous context: {' '.join(conversation_history[-3:])} Current question: {question}"
            else:
                context_question = question

            print(f"\n🤖 Consultant:")
            print("-" * 20)

            start_time = time.time()
            response = scenario_customer_consultation(context_question)
            end_time = time.time()

            conversation_history.append(f"Q: {question}")
            conversation_history.append(f"A: {str(response)[:100]}")

            print(f"\n⏱️ Response time: {end_time - start_time:.2f}s")
            print(f"💬 Conversation length: {len(conversation_history)//2} exchanges")
            print("-" * 50)

        except KeyboardInterrupt:
            break

    print(f"\n📊 Conversation ended. Total exchanges: {len(conversation_history)//2}")

# =========================================================================
# MAIN INTERACTIVE MENU
# =========================================================================

def interactive_menu():
    """Main interactive menu"""
    print("\n🎮 INTERACTIVE CONSULTANT TESTER")
    print("=" * 50)

    modes = {
        "1": ("Simple Chat", chat_with_consultant),
        "2": ("Guided Testing", guided_consultant_test),
        "3": ("Batch Testing", batch_test_mode),
        "4": ("Conversation Mode", conversation_mode),
        "5": ("Show Examples", show_example_questions)
    }

    while True:
        print("\n📋 Choose testing mode:")
        for key, (name, _) in modes.items():
            print(f"   {key}. {name}")
        print("   0. Exit")

        try:
            choice = input("\nEnter mode number (0-5): ").strip()

            if choice == "0":
                print("👋 Thank you for testing! Goodbye!")
                break
            elif choice in modes:
                mode_name, mode_function = modes[choice]
                print(f"\n🚀 Starting {mode_name}...")
                mode_function()
            else:
                print("⚠️ Invalid choice. Please enter 0-5")

        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")

# =========================================================================
# QUICK START FUNCTIONS
# =========================================================================

def start_chat():
    """Quick start for simple chat"""
    chat_with_consultant()

def start_guided():
    """Quick start for guided testing"""
    guided_consultant_test()

def start_interactive():
    """Quick start for full interactive menu"""
    interactive_menu()

# =========================================================================
# USAGE INSTRUCTIONS
# =========================================================================

print("""
🎯 INTERACTIVE TESTING OPTIONS:

🚀 QUICK START:
- start_chat() - Simple question-answer chat
- start_guided() - Category-based guided testing
- start_interactive() - Full interactive menu

💬 TESTING MODES:
1. Simple Chat - Ask any question, get instant answers
2. Guided Testing - Choose categories, get prompts
3. Batch Testing - Test multiple questions at once
4. Conversation Mode - Extended dialogues with context
5. Examples - See sample questions for inspiration

⌨️ COMMANDS IN CHAT:
- Type your question normally
- 'help' - Show example questions
- 'quit'/'exit'/'stop' - End session
- Ctrl+C - Emergency exit

💡 TIPS:
- Start with start_chat() for immediate testing
- Use guided mode to explore different question types
- Batch mode is great for systematic testing
- Conversation mode tests context awareness

🎮 Ready to start interactive testing!
Type: start_chat() to begin
""")

In [ ]:
start_chat()

In [ ]:

import subprocess
import threading
import time
import requests

print("🔍 Проверяем статус Ollama сервера...")

# Проверяем, работает ли сервер
try:
    response = requests.get("http://127.0.0.1:11434", timeout=5)
    print("✅ Ollama сервер уже запущен!")
except:
    print("❌ Ollama сервер не отвечает. Запускаем...")

    # Запускаем сервер в фоновом режиме
    def run_ollama():
        subprocess.Popen(["ollama", "serve"])

    thread = threading.Thread(target=run_ollama)
    thread.daemon = True  # Демон поток
    thread.start()

    print("⏳ Ждем запуска сервера...")
    time.sleep(10)  # Даем время на запуск

    # Проверяем снова
    try:
        response = requests.get("http://127.0.0.1:11434", timeout=10)
        print("✅ Ollama сервер успешно запущен!")
    except:
        print("❌ Не удалось запустить сервер. Попробуем альтернативный способ...")
